In [1]:
#library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as ap

In [2]:
#mount file from google drive
from google.colab import drive
drive.mount('/content/drive')
#grab the data
img224 = np.load('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/img224.npy')
label  = np.load('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/label.npy')

Mounted at /content/drive


In [3]:
X224_train = img224
y224_train = label
y224_train = tf.keras.utils.to_categorical(y224_train, 4)

In [4]:
#checkpoint
checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/FINAL_DL/checkpoint/vgg2'
model_checkpoint_callack = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weight_only = True,
    save_freq = 'epoch',
    mode = 'auto',
    save_best_only = True,
    monitor = 'val_accuracy'
)

In [5]:
#VGG16
def model_vgg16(X_train, y_train,epochs = 8):
    base_model = ap.VGG16(weights='imagenet',include_top=False, input_shape=(224,224,3))
    base_model.trainable = False

    head_model = base_model.output
    head_model = keras.layers.Flatten()(head_model)
    #head_model = keras.layers.Dense(512, activation = 'relu')(head_model)
    #head_model = keras.layers.Dropout(0.5)(head_model)
    #head_model = keras.layers.Dense(128, activation = 'relu')(head_model)
    head_model = keras.layers.Dense(4,activation='softmax')(head_model)

    model = keras.Model(base_model.input,head_model)

    callbacks = [keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/Colab Notebooks/FINAL_DL/logs/vgg2'),model_checkpoint_callack]
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(X_train,y_train,batch_size=32, epochs=epochs, callbacks = callbacks, validation_split = 0.2)
    #unfreeze
    base_model.trainable = False
    model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy',metrics=['accuracy'])
    history = model.fit(X_train,y_train,batch_size=32, epochs=epochs, callbacks = callbacks, validation_split = 0.2)
    return model,history

In [6]:
model_vgg16_m3,history= model_vgg16(X224_train,y224_train,10)

58900480/58889256 [==============================] - 0s 0us/step
Epoch 1/10
75/75 [==============================] - 98s 1s/step - loss: 0.8785 - accuracy: 0.6771 - val_loss: 0.5197 - val_accuracy: 0.8183
Epoch 2/10
75/75 [==============================] - 91s 1s/step - loss: 0.3397 - accuracy: 0.8825 - val_loss: 0.4715 - val_accuracy: 0.8017
Epoch 3/10
75/75 [==============================] - 96s 1s/step - loss: 0.2298 - accuracy: 0.9225 - val_loss: 0.3294 - val_accuracy: 0.8833
Epoch 4/10
75/75 [==============================] - 100s 1s/step - loss: 0.1923 - accuracy: 0.9371 - val_loss: 0.2755 - val_accuracy: 0.9050
Epoch 5/10
75/75 [==============================] - 101s 1s/step - loss: 0.1258 - accuracy: 0.9642 - val_loss: 0.2700 - val_accuracy: 0.8983
Epoch 6/10
75/75 [==============================] - 104s 1s/step - loss: 0.0940 - accuracy: 0.9771 - val_loss: 0.2529 - val_accuracy: 0.9200
Epoch 7/10
75/75 [==============================] - 101s 1s/step - loss: 0.0718 - accuracy: 

In [7]:
model_vgg16_m3.save_weights('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/weights/vgg_m3_weights.h5')

In [9]:
#save the curve
his = pd.DataFrame(history.history)
his.to_pickle('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/curve/history_vgg3.pkl')

In [ ]:
model_vgg16_m3.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   